In [1]:
import tensorflow as tf
save_file = './save_W_B/model.ckpt'

weights = tf.Variable(tf.truncated_normal([2,3]),tf.float32)
bias = tf.Variable(tf.truncated_normal([3]),tf.float32)

saver = tf.train.Saver()
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print("weights:", sess.run(weights))
    print("bias:", sess.run(bias))
    saver.save(sess, save_file)
    print("Save to file:", save_file)

weights: [[ 1.2867733   0.63515556  0.6848697 ]
 [-0.24838382  0.7917348   0.14795004]]
bias: [ 0.43954504  0.8937031  -1.165228  ]
Save to file: ./save_W_B/model.ckpt


In [2]:
tf.reset_default_graph()

weights = tf.Variable(tf.truncated_normal([2,3]))
bias = tf.Variable(tf.truncated_normal([3]))

saver = tf.train.Saver()

# not need init step

with tf.Session() as sess:
    saver.restore(sess, save_file)
    print("weights:", sess.run(weights))
    print("bias:", sess.run(bias))


INFO:tensorflow:Restoring parameters from ./save_W_B/model.ckpt
weights: [[ 1.2867733   0.63515556  0.6848697 ]
 [-0.24838382  0.7917348   0.14795004]]
bias: [ 0.43954504  0.8937031  -1.165228  ]


In [16]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import tensorflow as tf
import math

tf.reset_default_graph()


learning_rate = 0.001
n_input = 784  # MNIST 数据输入 (图片尺寸: 28*28)
n_classes = 10  # MNIST 总计类别 (数字 0-9)

mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)

features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

logits = tf.add(tf.matmul(features, weights), bias)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

save_file = './save_model/train_model.ckpt'
batch_size = 128
n_epochs = 100

print(mnist.train.num_examples / batch_size, math.ceil(mnist.train.num_examples / batch_size))

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={features: batch_features, labels: batch_labels})

        if epoch % 10 == 0:
                valid_accuracy = sess.run(
                    accuracy,
                    feed_dict={
                        features: mnist.validation.images,
                        labels: mnist.validation.labels})
                print('Epoch {:<3} - Validation Accuracy: {}'.format(epoch, valid_accuracy))
    saver.save(sess, save_file)
    print("Trained Model Saved")

Extracting /datasets/ud730/mnist\train-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist\train-labels-idx1-ubyte.gz
Extracting /datasets/ud730/mnist\t10k-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist\t10k-labels-idx1-ubyte.gz
429.6875 430
Epoch 0   - Validation Accuracy: 0.13379999995231628
Epoch 10  - Validation Accuracy: 0.28119999170303345
Epoch 20  - Validation Accuracy: 0.41519999504089355
Epoch 30  - Validation Accuracy: 0.5019999742507935
Epoch 40  - Validation Accuracy: 0.5648000240325928
Epoch 50  - Validation Accuracy: 0.6093999743461609
Epoch 60  - Validation Accuracy: 0.642799973487854
Epoch 70  - Validation Accuracy: 0.6704000234603882
Epoch 80  - Validation Accuracy: 0.6901999711990356
Epoch 90  - Validation Accuracy: 0.7075999975204468
Trained Model Saved


In [15]:
saver = tf.train.Saver() 
with tf.Session() as sess:
    saver.restore(sess, save_file)
    
    test_accuracy = sess.run(accuracy, feed_dict={features: mnist.test.images, labels: mnist.test.labels})
print("Test Accuracy:{}".format(test_accuracy))
    

INFO:tensorflow:Restoring parameters from ./save_model/train_model.ckpt
Test Accuracy:0.7159000039100647


In [24]:
import tensorflow as tf

hidden_layer_weights = [
    [0.1, 0.2, 0.4],
    [0.4, 0.6, 0.6],
    [0.5, 0.9, 0.1],
    [0.8, 0.2, 0.8]]
out_weights = [
    [0.1, 0.6],
    [0.2, 0.1],
    [0.7, 0.9]]

# Weights and biases
weights = [tf.Variable(hidden_layer_weights), tf.Variable(out_weights)]
biases = [tf.Variable(tf.zeros(3)), tf.Variable(tf.zeros(2))]

features = tf.Variable([[0.0, 2.0, 3.0, 4.0], [0.1, 0.2, 0.3, 0.4], [11.0, 12.0, 13.0, 14.0]])

keep_prob = tf.placeholder(tf.float32)
hidden_layer = tf.add(tf.matmul(features, weights[0]), biases[0])
hidden_layer = tf.nn.relu(hidden_layer)
hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)

logits = tf.add(tf.matmul(hidden_layer, weights[1]), biases[1])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(logits, feed_dict={keep_prob: 0.5}))

[[ 1.88        0.94      ]
 [ 0.71400005  0.91800004]
 [14.280001   33.1       ]]


In [35]:
k_output = 64
image_width = 10
image_height = 10
color_channels = 3

filter_size_width = 5
filter_size_height = 5

input = tf.placeholder(tf.float32, shape = [None, image_height, image_width, color_channels])
weight = tf.Variable(tf.truncated_normal([filter_size_height, filter_size_width, color_channels, k_output]))
bias = tf.Variable(tf.zeros(k_output))
print(input.shape, weight.shape, bias.shape)
conv_layer = tf.nn.conv2d(input, weight, strides = [1,2,2,1], padding = "SAME")
conv_layer = tf.nn.bias_add(conv_layer, bias)
conv_layer = tf.nn.relu(conv_layer)

(?, 10, 10, 3) (5, 5, 3, 64) (64,)
